In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
from Bio import SeqIO
from rdkit import Chem
from rdkit.Chem import AllChem
from IPython.display import display

cwd = os.getcwd()
if cwd.endswith('notebook/PG'):
    os.chdir('../..')
    cwd = os.getcwd()

from src.pg_modelling.ligand_utils import generate_ccd_from_smiles, sanitize_ligand_name, generate_conformation

In [2]:
data_folder = Path(os.path.expanduser('~')) / 'Documents' / 'PG_modelling'
assert data_folder.is_dir()

In [3]:
glycan_modifications = pd.read_csv(
    data_folder / 'glycan_modifications' / 'glycan_modifications.csv', 
    index_col='ligand_name',
)
print(f'Number of fragments in subset: {len(glycan_modifications):,}')
glycan_modifications

Number of fragments in subset: 9


,Name,Synonym,Formula,Monoisotopic Mass,Modifications,Degree Amidation,Degree Acetylation,Ontology,PGN Units,Glycan Units,...,Peptide,SMILES,INCHIKEY,clogP,RT,[M+H]+,[M+Na]+,[M+K]+,[M+2H]2+,[M+3H]3+
ligand_name,,,,,,,,,,,,,,,,,,,,,
G-NAM,(G)(NAM)-None,GlcN.MurNAc--None,C17H30N2O12,454.17987,Amidase,0,1,G2,1,2,...,NaN,CC(=O)NC1C(O)OC(CO)C(OC2OC(CO)C(O)C(O)C2N)C1OC...,CCRWBXATDKHQDG-UHFFFAOYSA-N,-4.7896,0,455.18715,477.16909,493.14303,228.09722,152.40057
G-DAM,(G)(DAM)-None,GlcN.MurNAcOAc--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(O)OC(COC(C)=O)C(OC2OC(CO)C(O)C(O)C2...,WMTUYCBSSKTNNR-UHFFFAOYSA-N,-4.2188,0,497.19772,519.17966,535.15360,249.10250,166.40409
NAG-MUR,(NAG)(MUR)-None,GlcNAc.MurN--None,C17H30N2O12,454.17987,Amidase,0,1,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)O)OC(C...,IGYLVGQPIFKPAO-UHFFFAOYSA-N,-4.7896,0,455.18715,477.16909,493.14303,228.09722,152.40057
DAG-MUR,(DAG)(MUR)-None,GlcNAcOAc.MurN--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(N)C2OC(C)C(=O)O)OC(C...,ZSDDRWVIQWSQPJ-UHFFFAOYSA-N,-4.2188,0,497.19772,519.17966,535.15360,249.10250,166.40409
NAG-NAM,(NAG)(NAM)-None,GlcNAc.MurNAc--None,C19H32N2O13,496.19044,Amidase,0,2,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,WQKIVDUWFNRJHE-UHFFFAOYSA-N,-4.6122,0,497.19772,519.17966,535.15360,249.10250,166.40409
DAG-NAM,(DAG)(NAM)-None,GlcNAcOAc.MurNAc--None,C21H34N2O14,538.20100,Amidase,0,3,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(CO)OC(O)C(NC(C)=O)C2OC(C)C(=O)...,SUTALUMKMRSKIF-UHFFFAOYSA-N,-4.0414,0,539.20828,561.19022,577.16416,270.10778,180.40761
NAG-DAM,(NAG)(DAM)-None,GlcNAc.MurNAcOAc--None,C21H34N2O14,538.20100,Amidase,0,3,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,GKOAKWSMCSQKMH-UHFFFAOYSA-N,-4.0414,0,539.20828,561.19022,577.16416,270.10778,180.40761
DAG-DAM,(DAG)(DAM)-None,GlcNAcOAc.MurNAcOAc--None,C23H36N2O15,580.21157,Amidase,0,4,G2,1,2,...,NaN,CC(=O)NC1C(OC2C(COC(C)=O)OC(O)C(NC(C)=O)C2OC(C...,UGHTWFWJTFCKTB-UHFFFAOYSA-N,-3.4706,0,581.21885,603.20079,619.17473,291.11307,194.41114
G-MUR,(G)(MUR)-None,GlcN.MurN--None,C15H28N2O11,412.16931,Amidase,0,0,G2,1,2,...,NaN,CC(OC1C(N)C(O)OC(CO)C1OC1OC(CO)C(O)C(O)C1N)C(=O)O,UBADYJPVCNDSPI-UHFFFAOYSA-N,-4.9670,0,413.17659,435.15853,451.13247,207.09194,138.39705


In [4]:
ligand_folder = data_folder / 'glycan_modifications' / 'pdb'
ligand_folder.mkdir(exist_ok=True)

for ligand_name, row in glycan_modifications.iterrows():
    raw_name = row['Name']
    smiles = row['SMILES']

    mol = Chem.MolFromSmiles(smiles)
    try:
        mol = generate_conformation(mol)
    except ValueError:
        print(f'Error for ligand: {raw_name}')
        raise

    output_path = ligand_folder / f'{ligand_name}.pdb'
    Chem.MolToPDBFile(mol, output_path.as_posix())

[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling AddHs()
[21:52:24] Molecule does not have explicit Hs. Consider calling 